### This example shows how to use read/write functions for <ul>
<li>Input </li>
<li>Grid </li>
<li>Mask </li>
<li>Dep </li>
<li>Forcing </li>
</ul>

In [ ]:
# for first run install package
# pip install -e ..

In [ ]:
import os
import sys
from hydromt_sfincs import SfincsModel

### Input

In [ ]:
# initialize
model_root = "sfincs_coastal"
sf = SfincsModel(root=model_root, mode="r+")  # append mode

# read inp
sf.read_config()
sf.config

In [ ]:
# eventually change input and write again
sf.set_config("zsini", 0.0)
sf.write_config("sfincs_v2.inp")

### Grid

In [ ]:
# grid is automatically read in from sfincs.inp
reggrid = sf.reggrid
reggrid.__dict__

Compare different defenitions of grid

In [ ]:
# #Compare with previous definition of grid
# import math
# import numpy as np

# #old defenitions
# cosrot = math.cos(inp.rotation * math.pi / 180)
# sinrot = math.sin(inp.rotation * math.pi / 180)

# xx = np.linspace(
#     0.5 * inp.dx, inp.mmax * inp.dx - 0.5 * inp.dx, num=inp.mmax
# )
# yy = np.linspace(
#     0.5 * inp.dy, inp.nmax * inp.dy - 0.5 * inp.dy, num=inp.nmax
# )

# xg0, yg0 = np.meshgrid(xx, yy)
# xg = inp.x0 + xg0 * cosrot - yg0 * sinrot
# yg = inp.y0 + xg0 * sinrot + yg0 * cosrot

# #new defenitions
# x = grid.coordinates["x"]
# y = grid.coordinates["y"]

# xn,yn = np.meshgrid(x,y)

# #differences
# print(sum(sum(xg-xn)))
# print(sum(sum(yg-yn)))

### Mask

In [ ]:
# read maska

sf.read_grid(data_vars=["msk"])
# sf.read_grid()  #read all maps
sf.mask.plot.imshow()

In [ ]:
# write
sf.config.update(indexfile="sfincs_v2.ind", mskfile="sfincs_v2.msk")
sf.write_grid(data_vars=["msk"])  # index file is always written
# sf.write_grid()  # write all maps in sf.grid dataset
os.path.isfile(os.path.join(model_root, "sfincs_v2.msk"))

### Dep and Manning

In [ ]:
sf.read_grid(["dep"])
sf.grid["dep"].raster.mask_nodata().plot.imshow()

In [ ]:
# change depth and write away
import xarray as xr

sf.grid["dep"] = xr.where(
    sf.grid["dep"] != sf.grid["dep"].raster.nodata, sf.grid["dep"] + 10, -9999
)
sf.grid["dep"].raster.set_nodata(-9999)
sf.config.update(
    depfile="sfincs_v2.dep",
)
sf.write_grid(["dep"])

In [ ]:
# read again and plot
# NOTE now we read sfincs_v2.dep
sf.read_grid("dep")
sf.grid["dep"].raster.mask_nodata().plot.imshow()

In [ ]:
# FIXME file is called sfincs.manning
# man = sf.read_grid("manning")
# sf.grid["manning"].raster.mask_nodata().plot.imshow()

### Forcing

In [ ]:
# TODO continue from here
sf.read_forcing(["waterlevel"])

In [ ]:
# boundary points and time_series combined
sf.forcing["bzs"]

In [ ]:
# boundary timeseries
sf.plot_forcing()